In [36]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM
from keras.preprocessing.text import Tokenizer, one_hot
from keras.optimizers import RMSprop
#from google.colab import drive
import numpy as np
from data_utils import *
from music_utils import *

import numpy as np
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

In [2]:
import pandas as pd
import numpy as np
with open("t.txt","w")as f:
  f.write("hello")
data=pd.read_csv("MuseData_test_30.csv")

In [3]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,next_note
0,48,55,63,72,48,63,67,46,62,67,...,72,43,57,67,72,42,57,69,74,42
1,72,48,63,67,46,62,67,45,60,67,...,67,72,42,57,69,74,42,57,62,74
2,67,46,62,67,45,60,67,72,45,58,...,57,69,74,42,57,62,74,43,57,62
3,67,45,60,67,72,45,58,67,72,45,...,42,57,62,74,43,57,62,70,43,55
4,67,72,45,58,67,72,45,57,66,72,...,74,43,57,62,70,43,55,62,70,48


In [4]:
sub_sequences= data.iloc[:,:30].values
next_steps=data.iloc[:,30].values

In [6]:
len_notes=88
maxlen=30

X = np.zeros((len(sub_sequences), maxlen, len_notes), dtype=np.uint8 )
Y = np.zeros((len(sub_sequences), len_notes), dtype=np.uint8)
for i, seq in enumerate(sub_sequences):
    #print(seq)
    for t, note in enumerate(seq):
        X[i, t, note-21] = 1
        Y[i, note-21] = 1
print("x shape:",np.array(X).shape)
             

x shape: (69421, 30, 88)


In [7]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM
from keras.preprocessing.text import Tokenizer, one_hot
from keras.optimizers import RMSprop
#from google.colab import drive
import numpy as np
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

char_rnn = Sequential()
char_rnn.add(LSTM(32, input_shape=(maxlen, 88)))
char_rnn.add(Dense(88,activation="softmax"))

In [8]:

#char_rnn.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01),metrics=["acc"])
#char_rnn.fit(X,Y, epochs=50, batch_size=512)

In [9]:
## LSTM
len_notes=88
maxlen=30

X = np.zeros((len(sub_sequences), maxlen, len_notes), dtype=np.uint8 )
Y2 = np.zeros((maxlen, len(sub_sequences),  len_notes), dtype=np.uint8)
for i, seq in enumerate(sub_sequences):
    #print(seq)
    for t, note in enumerate(seq):
        

        if t>0:
            X[i, t, note-21] = 1
            Y2[t-1, i, note-21] = 1
print("x shape:",np.array(X).shape)
print("y2 shape:",np.array(Y2).shape)

x shape: (69421, 30, 88)
y2 shape: (30, 69421, 88)


In [43]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM
from keras.preprocessing.text import Tokenizer, one_hot
from keras.optimizers import RMSprop
#from google.colab import drive
import numpy as np
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K


n_a = 64 
n_values=88
reshapor = Reshape((1, 88))                        # Used in Step 2.B of djmodel(), below
LSTM_cell = LSTM(n_a,return_state=True)         # Used in Step 2.C
densor = Dense(n_values, activation='softmax')     # Used in Step 2.D

In [44]:


# GRADED FUNCTION: djmodel

def djmodel(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data 
    
    Returns:
    model -- a keras model with the 
    """
    
    # Define the input of your model with a shape 
    X = Input(shape=(Tx, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    ### START CODE HERE ### 
    # Step 1: Create empty list to append the outputs while you iterate (≈1 line)
    outputs = []
    
    # Step 2: Loop
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = Lambda(lambda x: X[:,t,:])(X) #Wraps arbitrary expression as a Layer object.
        # Step 2.B: Use reshapor to reshape x to be (1, n_values) (≈1 line)
        x =reshapor(x)    
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        #LSTM_cell(x, initial_state=[a, c])
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        # Step 2.E: add the output to "outputs"
        outputs.append(out)
        
    # Step 3: Create model instance
    model = Model(inputs = [X,a0,c0],outputs = outputs)
    
    ### END CODE HERE ###
    
    return model
model = djmodel(Tx = 30 , n_a = 64, n_values = 88)

In [45]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 30, 88)       0                                            
__________________________________________________________________________________________________
lambda_61 (Lambda)              (None, 88)           0           input_4[0][0]                    
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 1, 88)        0           lambda_61[0][0]                  
                                                                 lambda_62[0][0]                  
                                                                 lambda_63[0][0]                  
                                                                 lambda_64[0][0]                  
          

#### Adam

In [46]:
opt = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
m = 69421
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

history=model.fit([X, a0, c0], list(Y2), epochs=20)

Epoch 1/20
69421/69421 [==============================] - 99s 1ms/step - loss: 60.1956 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0604 - dense_2_acc_1: 0.2395 - dense_2_acc_2: 0.2839 - dense_2_acc_3: 0.3220 - dense_2_acc_4: 0.3763 - dense_2_acc_5: 0.4133 - dense_2_acc_6: 0.4404 - dense_2_acc_7: 0.4583 - dense_2_acc_8: 0.4676 - dense_2_acc_9: 0.4779 - dense_2_acc_10: 0.4866 - dense_2_acc_11: 0.4900 - dense_2_acc_12: 0.4903 - dense_2_acc_13: 0.4937 - dense_2_acc_14: 0.4910 - dense_2_acc_15: 0.4925 - dense_2_acc_16: 0.4953 - dense_2_acc_17: 0.4929 - dense_2_acc_18: 0.4934 - dense_2_acc_19: 0.4968 - dense_2_acc_20: 0.4949 - dense_2_acc_21: 0.4946 - dense_2_acc_22: 0.4966 - dense_2_acc_23: 0.4950 - dense_2_acc_24: 0.4933 - dense_2_acc_25: 0.4952 - dense_2_acc_26: 0.4941 - dense_2_acc_27: 0.4932 - dense_2_acc_28: 0.4958 - dense_2_acc_29: 0.0000e+00
Epoch 2/20
69421/69421 [==============================] - 75s 1ms/step - loss: 54.0141 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0616 - dense_

69421/69421 [==============================] - 78s 1ms/step - loss: 51.9263 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0640 - dense_2_acc_1: 0.2432 - dense_2_acc_2: 0.2943 - dense_2_acc_3: 0.3437 - dense_2_acc_4: 0.4105 - dense_2_acc_5: 0.4584 - dense_2_acc_6: 0.4935 - dense_2_acc_7: 0.5230 - dense_2_acc_8: 0.5375 - dense_2_acc_9: 0.5532 - dense_2_acc_10: 0.5663 - dense_2_acc_11: 0.5706 - dense_2_acc_12: 0.5742 - dense_2_acc_13: 0.5811 - dense_2_acc_14: 0.5794 - dense_2_acc_15: 0.5812 - dense_2_acc_16: 0.5871 - dense_2_acc_17: 0.5834 - dense_2_acc_18: 0.5836 - dense_2_acc_19: 0.5884 - dense_2_acc_20: 0.5856 - dense_2_acc_21: 0.5852 - dense_2_acc_22: 0.5886 - dense_2_acc_23: 0.5859 - dense_2_acc_24: 0.5853 - dense_2_acc_25: 0.5892 - dense_2_acc_26: 0.5862 - dense_2_acc_27: 0.5849 - dense_2_acc_28: 0.5890 - dense_2_acc_29: 0.0000e+00TA: 1:13 - loss: 51.7966 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0615 - dense_2_acc_1: 0.2426 - dense_2_acc_2: 0.3249 - dense_2_acc_3: 0.3407 - dense_2_

69421/69421 [==============================] - 76s 1ms/step - loss: 51.6222 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0643 - dense_2_acc_1: 0.2429 - dense_2_acc_2: 0.2952 - dense_2_acc_3: 0.3441 - dense_2_acc_4: 0.4114 - dense_2_acc_5: 0.4607 - dense_2_acc_6: 0.4952 - dense_2_acc_7: 0.5247 - dense_2_acc_8: 0.5398 - dense_2_acc_9: 0.5549 - dense_2_acc_10: 0.5694 - dense_2_acc_11: 0.5727 - dense_2_acc_12: 0.5767 - dense_2_acc_13: 0.5827 - dense_2_acc_14: 0.5826 - dense_2_acc_15: 0.5839 - dense_2_acc_16: 0.5893 - dense_2_acc_17: 0.5870 - dense_2_acc_18: 0.5873 - dense_2_acc_19: 0.5911 - dense_2_acc_20: 0.5877 - dense_2_acc_21: 0.5890 - dense_2_acc_22: 0.5925 - dense_2_acc_23: 0.5888 - dense_2_acc_24: 0.5886 - dense_2_acc_25: 0.5935 - dense_2_acc_26: 0.5890 - dense_2_acc_27: 0.5876 - dense_2_acc_28: 0.5919 - dense_2_acc_29: 1.4405e-05: 42s - loss: 51.6561 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0638 - dense_2_acc_1: 0.2435 - dense_2_acc_2: 0.2959 - dense_2_acc_3: 0.3446 - dense_2_acc

69421/69421 [==============================] - 73s 1ms/step - loss: 51.3403 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0641 - dense_2_acc_1: 0.2424 - dense_2_acc_2: 0.2951 - dense_2_acc_3: 0.3442 - dense_2_acc_4: 0.4121 - dense_2_acc_5: 0.4615 - dense_2_acc_6: 0.4973 - dense_2_acc_7: 0.5258 - dense_2_acc_8: 0.5421 - dense_2_acc_9: 0.5580 - dense_2_acc_10: 0.5721 - dense_2_acc_11: 0.5757 - dense_2_acc_12: 0.5800 - dense_2_acc_13: 0.5863 - dense_2_acc_14: 0.5857 - dense_2_acc_15: 0.5875 - dense_2_acc_16: 0.5924 - dense_2_acc_17: 0.5893 - dense_2_acc_18: 0.5908 - dense_2_acc_19: 0.5938 - dense_2_acc_20: 0.5911 - dense_2_acc_21: 0.5915 - dense_2_acc_22: 0.5963 - dense_2_acc_23: 0.5918 - dense_2_acc_24: 0.5917 - dense_2_acc_25: 0.5957 - dense_2_acc_26: 0.5920 - dense_2_acc_27: 0.5911 - dense_2_acc_28: 0.5955 - dense_2_acc_29: 1.4405e-05
Epoch 12/20
69421/69421 [==============================] - 76s 1ms/step - loss: 51.2725 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0645 - dense_2_acc_1: 0

69421/69421 [==============================] - 76s 1ms/step - loss: 50.9560 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0644 - dense_2_acc_1: 0.2430 - dense_2_acc_2: 0.2951 - dense_2_acc_3: 0.3452 - dense_2_acc_4: 0.4133 - dense_2_acc_5: 0.4627 - dense_2_acc_6: 0.4978 - dense_2_acc_7: 0.5278 - dense_2_acc_8: 0.5451 - dense_2_acc_9: 0.5603 - dense_2_acc_10: 0.5744 - dense_2_acc_11: 0.5778 - dense_2_acc_12: 0.5827 - dense_2_acc_13: 0.5891 - dense_2_acc_14: 0.5887 - dense_2_acc_15: 0.5916 - dense_2_acc_16: 0.5969 - dense_2_acc_17: 0.5948 - dense_2_acc_18: 0.5956 - dense_2_acc_19: 0.5992 - dense_2_acc_20: 0.5967 - dense_2_acc_21: 0.5968 - dense_2_acc_22: 0.6010 - dense_2_acc_23: 0.5965 - dense_2_acc_24: 0.5962 - dense_2_acc_25: 0.6008 - dense_2_acc_26: 0.5964 - dense_2_acc_27: 0.5955 - dense_2_acc_28: 0.6006 - dense_2_acc_29: 2.8810e-05: 19s - loss: 50.9153 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0640 - dense_2_acc_1: 0.2437 - dense_2_acc_2: 0.2953 - dense_2_acc_3: 0.3461 - dense_2_acc

In [48]:
# GRADED FUNCTION: music_inference_model

def music_inference_model(LSTM_cell, densor, n_values = 88, n_a = 64, Ty = 100):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_values -- integer, umber of unique values
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """

    # Define the input of your model with a shape 
    x0 = Input(shape=(1, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0

    ### START CODE HERE ###
    # Step 1: Create an empty list of "outputs" to later store your predicted values (≈1 line)
    outputs =[]
    
    # Step 2: Loop over Ty and generate a value at every time step
    for t in range(Ty):
        #print(t)
        # Step 2.A: Perform one step of LSTM_cell (≈1 line)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        
        # Step 2.B: Apply Dense layer to the hidden state output of the LSTM_cell (≈1 line)
        out = densor(a)

        # Step 2.C: Append the prediction "out" to "outputs". out.shape = (None, 78) (≈1 line)
        outputs.append(out)
        print(out)
        
        # Step 2.D: Select the next value according to "out", and set "x" to be the one-hot representation of the
        #           selected value, which will be passed as the input to LSTM_cell on the next step. We have provided 
        #           the line of code you need to do this. 
       # x = Lambda(one_hot)(out)
       # x = Lambda(one_hot)(1)
     #   out=K.cast(o,"int32")
      #  x= Lambda(K.one_hot,
        #       arguments={'num_classes':88},
         #      output_shape=(1,1))(out)
        x=reshapor(out)
    
        
    # Step 3: Create model instance with the correct "inputs" and "outputs" (≈1 line)
    inference_model = Model(inputs = [x0,a0,c0],outputs = outputs)
    
    ### END CODE HERE ###
    
   # return inference_model
    return inference_model

In [49]:
inference_model = music_inference_model(LSTM_cell, densor, n_values = 88, n_a = 64, Ty = 60)
x_initializer = np.zeros((1, 1, 88))
#x_initializer+=28
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

Tensor("dense_2_30/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_31/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_32/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_33/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_34/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_35/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_36/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_37/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_38/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_39/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_40/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_41/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_42/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_43/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_44/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_45/Softmax:0", shape=(?, 88), dtype=float32)
Tensor("dense_2_46/Softm

In [50]:
# GRADED FUNCTION: predict_and_sample

def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Predicts the next value of values using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference time
    x_initializer -- numpy array of shape (1, 1, 78), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    
    Returns:
    results -- numpy-array of shape (Ty, 78), matrix of one-hot vectors representing the values generated
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
      ### START CODE HERE ###
    # Step 1: Use your inference model to predict an output sequence given x_initializer, a_initializer and c_initializer.
    pred = inference_model.predict([x_initializer, a_initializer, c_initializer])
    # Step 2: Convert "pred" into an np.array() of indices with the maximum probabilities
    indices = np.argmax(pred,axis = -1)
    # Step 3: Convert indices to one-hot vectors, the shape of the results should be (1, )
    results =to_categorical(indices)
    
    return results, indices

#### Generate music and post-processing

In [51]:
from music_utils import * 
from preprocess import * 
from keras.utils import to_categorical

chords, abstract_grammars = get_musical_data('original_metheny.mid')
corpus, tones, tones_indices, indices_tones = get_corpus_data(abstract_grammars)
N_tones = len(set(corpus))
n_a = 64
x_initializer = np.zeros((1, 1, 88))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

def load_music_utils():
    chords, abstract_grammars = get_musical_data('original_metheny.mid')
    corpus, tones, tones_indices, indices_tones = get_corpus_data(abstract_grammars)
    N_tones = len(set(corpus))
    X, Y, N_tones = data_processing(corpus, tones_indices, 60, 30)   
    return (X, Y, N_tones, indices_tones)


def generate_music(inference_model, corpus = corpus, abstract_grammars = abstract_grammars, tones = tones, tones_indices = tones_indices, indices_tones = indices_tones, T_y = 10, max_tries = 1000, diversity = 0.5):
    """
    Generates music using a model trained to learn musical patterns of a jazz soloist. Creates an audio stream
    to save the music and play it.
    
    Arguments:
    model -- Keras model Instance, output of djmodel()
    corpus -- musical corpus, list of 193 tones as strings (ex: 'C,0.333,<P1,d-5>')
    abstract_grammars -- list of grammars, on element can be: 'S,0.250,<m2,P-4> C,0.250,<P4,m-2> A,0.250,<P4,m-2>'
    tones -- set of unique tones, ex: 'A,0.250,<M2,d-4>' is one element of the set.
    tones_indices -- a python dictionary mapping unique tone (ex: A,0.250,< m2,P-4 >) into their corresponding indices (0-77)
    indices_tones -- a python dictionary mapping indices (0-77) into their corresponding unique tone (ex: A,0.250,< m2,P-4 >)
    Tx -- integer, number of time-steps used at training time
    temperature -- scalar value, defines how conservative/creative the model is when generating music
    
    Returns:
    predicted_tones -- python list containing predicted tones
    """
    
    # set up audio stream
    out_stream = stream.Stream()
    
    # Initialize chord variables
    curr_offset = 0.0                                     # variable used to write sounds to the Stream.
    num_chords = int(len(chords) / 3)                     # number of different set of chords
    
    print("Predicting new values for different set of chords.")
    # Loop over all 18 set of chords. At each iteration generate a sequence of tones
    # and use the current chords to convert it into actual sounds 
    for i in range(1, num_chords):
        
        # Retrieve current chord from stream
        curr_chords = stream.Voice()
        
        # Loop over the chords of the current set of chords
        for j in chords[i]:
            # Add chord to the current chords with the adequate offset, no need to understand this
            curr_chords.insert((j.offset % 4), j)
        
        # Generate a sequence of tones using the model
        _, indices = predict_and_sample(inference_model)
        indices = list(indices.squeeze())
        pred = [indices_tones[p] for p in indices]
        
        predicted_tones = 'C,0.25 '
        for k in range(len(pred) - 1):
            predicted_tones += pred[k] + ' ' 
        
        predicted_tones +=  pred[-1]
                
        #### POST PROCESSING OF THE PREDICTED TONES ####
        # We will consider "A" and "X" as "C" tones. It is a common choice.
        predicted_tones = predicted_tones.replace(' A',' C').replace(' X',' C')

        # Pruning #1: smoothing measure
        predicted_tones = prune_grammar(predicted_tones)
        
        # Use predicted tones and current chords to generate sounds
        sounds = unparse_grammar(predicted_tones, curr_chords)

        # Pruning #2: removing repeated and too close together sounds
        sounds = prune_notes(sounds)

        # Quality assurance: clean up sounds
        sounds = clean_up_notes(sounds)

        # Print number of tones/notes in sounds
        print('Generated %s sounds using the predicted values for the set of chords ("%s") and after pruning' % (len([k for k in sounds if isinstance(k, note.Note)]), i))
        
        # Insert sounds into the output stream
        for m in sounds:
            out_stream.insert(curr_offset + m.offset, m)
        for mc in curr_chords:
            out_stream.insert(curr_offset + mc.offset, mc)

        curr_offset += 4.0
        
    # Initialize tempo of the output stream with 130 bit per minute
    out_stream.insert(0.0, tempo.MetronomeMark(number=130))

    # Save audio stream to fine
    mf = midi.translate.streamToMidiFile(out_stream)
    mf.open("output/my_music.mid", 'wb')
    mf.write()
    print("Your generated music is saved in output/my_music.mid")
    mf.close()
    
    # Play the final stream through output (see 'play' lambda function above)
    # play = lambda x: midi.realtime.StreamPlayer(x).play()
    # play(out_stream)
    
    return out_stream


def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Predicts the next value of values using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference time
    x_initializer -- numpy array of shape (1, 1, 78), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    Ty -- length of the sequence you'd like to generate.
    
    Returns:
    results -- numpy-array of shape (Ty, 78), matrix of one-hot vectors representing the values generated
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
    ### START CODE HERE ###
    print(c_initializer.shape)
    pred = inference_model.predict([x_initializer, a_initializer, c_initializer])
    indices = np.argmax(pred, axis = -1)
    results = to_categorical(indices, num_classes=88)
    ### END CODE HERE ###
    
    return results, indices
out_stream = generate_music(inference_model)

Predicting new values for different set of chords.
(1, 64)
Generated 61 sounds using the predicted values for the set of chords ("1") and after pruning
(1, 64)
Generated 61 sounds using the predicted values for the set of chords ("2") and after pruning
(1, 64)
Generated 61 sounds using the predicted values for the set of chords ("3") and after pruning
(1, 64)
Generated 60 sounds using the predicted values for the set of chords ("4") and after pruning
(1, 64)
Generated 61 sounds using the predicted values for the set of chords ("5") and after pruning
Your generated music is saved in output/my_music.mid
